In [2]:
using Revise
using ConvexHullPricing
using DataFrames
using Plots
using JLD2
using Statistics, JuMP, Gurobi

const UT = ConvexHullPricing.Utilitaries
const OPT = ConvexHullPricing.Optimizer

function mon(array)
    result = Float64[]
    last = array[1]
    for elt in array
      new_last = minimum([elt, last])
      push!(result, new_last)
      last = new_last
    end
    return result
end
function get_last_elements(array)
  percentage = 10 / 100
  last_index = Int(ceil(length(array) * percentage))
  last_elements = mean(array[last_index:end])
  return last_elements
end

get_last_elements (generic function with 1 method)

In [8]:
function load_belgian_up_to_t(max_T = 900)
    matrixBLM = []
    matrixBPLM = []
    matrixSUBG = []
    matrixSGEP = []
    matrixDA = []
    matrixDWG = []
    matrixFGM = []
    matrixSUBGL = []
    for idx=1:8
        FS = maximum(load_object("..//results//optimal_values//NewRefinedOptRunBE$(idx).jld2")[3])
        XBLM, ITBLM, BLM, TBLM = load_object("..//results//15min_runs//BundleLevelMethodBE$(idx).jld2")
        XBPLM, ITBPLM, BPLM, TBPLM = load_object("..//results//15min_runs//BundleProximalLevelMethod-L-BE$(idx).jld2")
        XSUBG, ITSUBG, SUBG, TSUBG = load_object("..//results//15min_runs//SubGBE$(idx).jld2")
        XSGEP, ITSGEP, SGEP, TSGEP = load_object("..//results//15min_runs//SubG-EP-BE$(idx).jld2")
        XDA, ITDA, DA, TDA = load_object("..//results//15min_runs//D-AdaptationBE$(idx).jld2")
        XDWG, ITDWG, DWG, TDWG = load_object("..//results//15min_runs//DowGBE$(idx).jld2")
        XFGM, ITFGM, FGM, TFGM = load_object("..//results//15min_runs//FGMBE$(idx).jld2")
        XSUBGL, ITSUBGL, SUBGL, TSUBGL = load_object("..//results//15min_runs//LSUBGBE$(idx).jld2")
        max_SUBGL = searchsortedfirst(TSUBGL[2:end], max_T)
        max_BLM = searchsortedfirst(TBLM[2:end], max_T)
        max_BPLM = searchsortedfirst(TBPLM[2:end], max_T)
        max_SUBG = searchsortedfirst(TSUBG[2:end], max_T)
        max_SGEP = searchsortedfirst(TSGEP[2:end], max_T)
        max_DA = searchsortedfirst(TDA[2:end], max_T)
        max_DWG = searchsortedfirst(TDWG[2:end], max_T)
        max_FGM = searchsortedfirst(TFGM[2:end], max_T)
        F0 = 0.
        BLM = mon(FS .- BLM[1:max_BLM]) ./ (FS - F0)
        BPLM = mon(FS .- BPLM[1:max_BPLM]) ./ (FS - F0)
        SUBG = mon(FS .- SUBG[1:max_SUBG]) ./ (FS - F0)
        SGEP = mon(FS .- SGEP[1:max_SGEP]) ./ (FS - F0)
        DA = mon(FS .- DA[1:max_DA]) ./ (FS - F0)
        DWG = mon(FS .- DWG[1:max_DWG]) ./ (FS - F0)
        FGM = mon(FS .- FGM[1:max_FGM]) ./ (FS - F0)
        SUBGL = mon(FS .- SUBGL[1:max_SUBGL]) ./ (FS - F0)
        push!(matrixBLM, BLM)
        push!(matrixBPLM, BPLM)
        push!(matrixSUBG, SUBG)
        push!(matrixSGEP, SGEP)
        push!(matrixDA, DA)
        push!(matrixDWG, DWG)
        push!(matrixFGM, FGM)
        push!(matrixSUBGL, SUBGL)
    end
    return (matrixBLM), (matrixBPLM), (matrixSUBG), (matrixSGEP), (matrixDA), (matrixDWG), (matrixFGM), (matrixSUBGL)
end
function load_californian_up_to_t(max_T = 900)
    matrixBLM = []
    matrixBPLM = []
    matrixSUBG = []
    matrixSGEP = []
    matrixDA = []
    matrixDWG = []
    matrixFGM = []
    matrixSUBGL = []
    for idx=1:20
        if idx ∉ [9 10 11 12]
        FS = maximum(load_object("..//results//optimal_values//NewRefinedOptRunCA$(idx).jld2")[3])
        XBLM, ITBLM, BLM, TBLM = load_object("..//results//15min_runs//BundleLevelMethodCA$(idx).jld2")
        XBPLM, ITBPLM, BPLM, TBPLM = load_object("..//results//15min_runs//BundleProximalLevelMethod-L-CA$(idx).jld2")
        XSUBG, ITSUBG, SUBG, TSUBG = load_object("..//results//15min_runs//SubGCA$(idx).jld2")
        XSGEP, ITSGEP, SGEP, TSGEP = load_object("..//results//15min_runs//SubG-EP-CA$(idx).jld2")
        XDA, ITDA, DA, TDA = load_object("..//results//15min_runs//D-AdaptationCA$(idx).jld2")
        XDWG, ITDWG, DWG, TDWG = load_object("..//results//15min_runs//DowGCA$(idx).jld2")
        XFGM, ITFGM, FGM, TFGM = load_object("..//results//15min_runs//FGMCA$(idx).jld2")
        XSUBGL, ITSUBGL, SUBGL, TSUBGL = load_object("..//results//15min_runs//LSUBGCA$(idx).jld2")
        max_SUBGL = searchsortedfirst(TSUBGL[2:end], max_T)
        max_BLM = searchsortedfirst(TBLM[2:end], max_T)
        max_BPLM = searchsortedfirst(TBPLM[2:end], max_T)
        max_SUBG = searchsortedfirst(TSUBG[2:end], max_T)
        max_SGEP = searchsortedfirst(TSGEP[2:end], max_T)
        max_DA = searchsortedfirst(TDA[2:end], max_T)
        max_DWG = searchsortedfirst(TDWG[2:end], max_T)
        max_FGM = searchsortedfirst(TFGM[2:end], max_T)
        
        FS = maximum([maximum(SUBGL), FS])
        F0 = 0.
        BLM = mon(FS .- BLM[1:max_BLM]) ./ (FS - F0)
        BPLM = mon(FS .- BPLM[1:max_BPLM]) ./ (FS - F0)
        SUBG = mon(FS .- SUBG[1:max_SUBG]) ./ (FS - F0)
        SGEP = mon(FS .- SGEP[1:max_SGEP]) ./ (FS - F0)
        DA = mon(FS .- DA[1:max_DA]) ./ (FS - F0)
        DWG = mon(FS .- DWG[1:max_DWG]) ./ (FS - F0)
        FGM = mon(FS .- FGM[1:max_FGM]) ./ (FS - F0)
        SUBGL = mon(FS .- SUBGL[1:max_SUBGL]) ./ (FS - F0)
        push!(matrixBLM, BLM)
        push!(matrixBPLM, BPLM)
        push!(matrixSUBG, SUBG)
        push!(matrixSGEP, SGEP)
        push!(matrixDA, DA)
        push!(matrixDWG, DWG)
        push!(matrixFGM, FGM)
        push!(matrixSUBGL, SUBGL)
        end
    end
    return (matrixBLM), (matrixBPLM), (matrixSUBG), (matrixSGEP), (matrixDA), (matrixDWG), (matrixFGM), (matrixSUBGL)
end

load_californian_up_to_t (generic function with 2 methods)

In [11]:
function load_avg_belgian()
  matrixBLM = []
  matrixBPLM = []
  matrixSUBG = []
  matrixSGEP = []
  matrixDA = []
  matrixDWG = []
  matrixFGM = []
  matrixSUBGL = []
  instances = []
  for file in readdir("..//data//belgian"; join=true)
    push!(instances, UT.load_data(file))
  end
  for idx=1:8
      instance = instances[idx]
      FS = maximum(load_object("..//results//optimal_values//NewRefinedOptRunBE$(idx).jld2")[3])
      XBLM, ITBLM, BLM, TBLM = load_object("..//results//15min_runs//BundleLevelMethodBE$(idx).jld2")
      XBPLM, ITBPLM, BPLM, TBPLM = load_object("..//results//15min_runs//BundleProximalLevelMethod-L-BE$(idx).jld2")
      XSUBG, ITSUBG, SUBG, TSUBG = load_object("..//results//15min_runs//SubGBE$(idx).jld2")
      XSGEP, ITSGEP, SGEP, TSGEP = load_object("..//results//15min_runs//SubG-EP-BE$(idx).jld2")
      XDA, ITDA, DA, TDA = load_object("..//results//15min_runs//D-AdaptationBE$(idx).jld2")
      XDWG, ITDWG, DWG, TDWG = load_object("..//results//15min_runs//DowGBE$(idx).jld2")
      XFGM, ITFGM, FGM, TFGM = load_object("..//results//15min_runs//FGMBE$(idx).jld2")
      XSUBGL, ITSUBGL, SUBGL, TSUBGL = load_object("..//results//15min_runs//LSUBGBE$(idx).jld2")

      
      avg_SUBGL = get_last_elements(ITSUBGL)
      avg_BLM = get_last_elements(ITBLM)
      avg_BPLM = get_last_elements(ITBPLM)
      avg_SUBG = get_last_elements(ITSUBG)
      avg_SGEP = get_last_elements(ITSGEP)
      avg_DA = get_last_elements(ITDA)
      avg_DWG = get_last_elements(ITDWG)
      avg_FGM = get_last_elements(ITFGM)

      F0 = 0.
      BLM = (FS - UT.exact_oracle(instance, avg_BLM)[1]) ./ (FS - F0)
      BPLM = (FS - UT.exact_oracle(instance, avg_BPLM)[1]) ./ (FS - F0)
      SUBG = (FS - UT.exact_oracle(instance, avg_SUBG)[1]) ./ (FS - F0)
      SGEP = (FS - UT.exact_oracle(instance, avg_SGEP)[1]) ./ (FS - F0)
      DA = (FS - UT.exact_oracle(instance, avg_DA)[1]) ./ (FS - F0)
      DWG = (FS - UT.exact_oracle(instance, avg_DWG)[1]) ./ (FS - F0)
      FGM = (FS - UT.exact_oracle(instance, avg_FGM)[1]) ./ (FS - F0)
      SUBGL = (FS - UT.exact_oracle(instance, avg_SUBGL)[1]) ./ (FS - F0)
  end
end

load_avg_belgian (generic function with 1 method)

In [12]:
BelgianBLM, BelgianBPLM, BelgianSUBG, BelgianSUBGEP, BelgianDA, BelgianDOWG, BelgianFGM, BelgianSUBGL = load_avg_belgian()

MethodError: MethodError: no method matching iterate(::Nothing)

Closest candidates are:
  iterate(!Matched::Base.AsyncGenerator, !Matched::Base.AsyncGeneratorState)
   @ Base asyncmap.jl:362
  iterate(!Matched::Base.AsyncGenerator)
   @ Base asyncmap.jl:362
  iterate(!Matched::MathOptInterface.Utilities.IndexMap, Any...)
   @ MathOptInterface C:\Users\Sofiane\.julia\packages\MathOptInterface\vrZOD\src\Utilities\copy\index_map.jl:104
  ...


In [4]:
# LOAD BELGIAN
BelgianBLM, BelgianBPLM, BelgianSUBG, BelgianSUBGEP, BelgianDA, BelgianDOWG, BelgianFGM, BelgianSUBGL = load_belgian_up_to_t(900)
CalifornianBLM, CalifornianBPLM, CalifornianSUBG, CalifornianSUBGEP, CalifornianDA, CalifornianDOWG, CalifornianFGM, CalifornianSUBGL = load_californian_up_to_t(900)

(Any[[1.671404382539058e-5, 1.671404382539058e-5, 1.671404382539058e-5, 1.671404382539058e-5, 1.671404382539058e-5, 1.671404382539058e-5, 1.671404382539058e-5, 1.671404382539058e-5, 1.671404382539058e-5, 1.671404382539058e-5  …  3.2188336147505555e-7, 3.2188336147505555e-7, 3.2188336147505555e-7, 3.2188336147505555e-7, 3.2188336147505555e-7, 3.2188336147505555e-7, 3.2188336147505555e-7, 3.2188336147505555e-7, 3.2188336147505555e-7, 3.2188336147505555e-7], [1.6714043827896756e-5, 1.6714043827896756e-5, 1.6714043827896756e-5, 1.6714043827896756e-5, 1.6714043827896756e-5, 1.6714043827896756e-5, 1.6714043827896756e-5, 1.6714043827896756e-5, 1.6714043827896756e-5, 1.6714043827896756e-5  …  3.2188331711492494e-7, 3.2188331711492494e-7, 3.2188331711492494e-7, 3.2188331711492494e-7, 3.2188331711492494e-7, 3.2188331711492494e-7, 3.2188331711492494e-7, 3.2188331711492494e-7, 3.2188331711492494e-7, 3.2188331711492494e-7], [1.67123654002135e-5, 1.67123654002135e-5, 1.67123654002135e-5, 1.671236540

In [13]:
arr = Float64[]
for (idx, inst) in enumerate(BelgianBLM)
    @show round(minimum(inst), sigdigits=2)
    push!(arr, minimum(inst))
end
@show std(arr)

round(minimum(inst), sigdigits = 2) = 9.7e-6
round(minimum(inst), sigdigits = 2) = 3.5e-6
round(minimum(inst), sigdigits = 2) = 1.7e-6
round(minimum(inst), sigdigits = 2) = 3.1e-6
round(minimum(inst), sigdigits = 2) = 3.5e-6
round(minimum(inst), sigdigits = 2) = 3.6e-6
round(minimum(inst), sigdigits = 2) = 4.2e-6
round(minimum(inst), sigdigits = 2) = 2.6e-6
std(arr) = 2.4289404248781557e-6


2.4289404248781557e-6

In [5]:
function load_belgian_time(eps)
    matrixBLM = []
    matrixBPLM = []
    matrixSUBG = []
    matrixSGEP = []
    matrixDA = []
    matrixDWG = []
    matrixFGM = []
    matrixSUBGL = []
    matrixCG = []
    for idx=1:8
        FS = maximum(load_object("..//results//optimal_values//NewRefinedOptRunBE$(idx).jld2")[3])
        XBLM, ITBLM, BLM, TBLM = load_object("..//results//15min_runs//BundleLevelMethodBE$(idx).jld2")
        XBPLM, ITBPLM, BPLM, TBPLM = load_object("..//results//15min_runs//BundleProximalLevelMethod-L-BE$(idx).jld2")
        XSUBG, ITSUBG, SUBG, TSUBG = load_object("..//results//15min_runs//SubGBE$(idx).jld2")
        XSGEP, ITSGEP, SGEP, TSGEP = load_object("..//results//15min_runs//SubG-EP-BE$(idx).jld2")
        XDA, ITDA, DA, TDA = load_object("..//results//15min_runs//D-AdaptationBE$(idx).jld2")
        XDWG, ITDWG, DWG, TDWG = load_object("..//results//15min_runs//DowGBE$(idx).jld2")
        XFGM, ITFGM, FGM, TFGM = load_object("..//results//15min_runs//FGMBE$(idx).jld2")
        XSUBGL, ITSUBGL, SUBGL, TSUBGL = load_object("..//results//15min_runs//LSUBGBE$(idx).jld2")
        XCG, ITCG, CG, TCG = load_object("..//results//15min_runs//ColumnGeneration-BE$(idx).jld2")
        max_SUBGL = searchsortedfirst(TSUBGL[2:end], 900)
        FS = maximum([maximum(SUBGL), maximum(CG), FS])
        push!(matrixBLM, change(BLM, TBLM, FS))
        push!(matrixBPLM, change(BPLM, TBPLM, FS))
        push!(matrixSUBG, change(SUBG, TSUBG, FS))
        push!(matrixSGEP, change(SGEP, TSGEP, FS))
        push!(matrixDA, change(DA, TDA, FS))
        push!(matrixDWG, change(DWG, TDWG, FS))
        push!(matrixFGM, change(FGM, TFGM, FS))
        push!(matrixCG, change(CG, TCG, FS))
        push!(matrixSUBGL, TSUBGL[searchsortedfirst((FS .- SUBGL[1:max_SUBGL]) ./ (FS), eps, rev=true)])
    end
    return (matrixBLM), (matrixBPLM), (matrixSUBG), (matrixSGEP), (matrixDA), (matrixDWG), (matrixFGM), (matrixSUBGL), matrixCG
end
function load_californian_time(eps)
    matrixBLM = []
    matrixBPLM = []
    matrixSUBG = []
    matrixSGEP = []
    matrixDA = []
    matrixDWG = []
    matrixFGM = []
    matrixSUBGL = []
    matrixCG = []
    for idx=1:20
        if idx ∉ [9 10 11 12]
        FS = maximum(load_object("..//results//optimal_values//NewRefinedOptRunCA$(idx).jld2")[3])
        XBLM, ITBLM, BLM, TBLM = load_object("..//results//15min_runs//BundleLevelMethodCA$(idx).jld2")
        XBPLM, ITBPLM, BPLM, TBPLM = load_object("..//results//15min_runs//BundleProximalLevelMethod-L-CA$(idx).jld2")
        XSUBG, ITSUBG, SUBG, TSUBG = load_object("..//results//15min_runs//SubGCA$(idx).jld2")
        XSGEP, ITSGEP, SGEP, TSGEP = load_object("..//results//15min_runs//SubG-EP-CA$(idx).jld2")
        XDA, ITDA, DA, TDA = load_object("..//results//15min_runs//D-AdaptationCA$(idx).jld2")
        XDWG, ITDWG, DWG, TDWG = load_object("..//results//15min_runs//DowGCA$(idx).jld2")
        XFGM, ITFGM, FGM, TFGM = load_object("..//results//15min_runs//FGMCA$(idx).jld2")
        XSUBGL, ITSUBGL, SUBGL, TSUBGL = load_object("..//results//15min_runs//LSUBGCA$(idx).jld2")
        XCG, ITCG, CG, TCG = load_object("..//results//15min_runs//ColumnGeneration-CA$(idx).jld2")
        max_SUBGL = searchsortedfirst(TSUBGL[2:end], 900)
        FS = maximum([maximum(SUBGL), maximum(CG), FS])
        push!(matrixBLM, change(BLM, TBLM, FS))
        push!(matrixBPLM, change(BPLM, TBPLM, FS))
        push!(matrixSUBG, change(SUBG, TSUBG, FS))
        push!(matrixSGEP, change(SGEP, TSGEP, FS))
        push!(matrixDA, change(DA, TDA, FS))
        push!(matrixDWG, change(DWG, TDWG, FS))
        push!(matrixFGM, change(FGM, TFGM, FS))
        push!(matrixCG, change(CG, TCG, FS))
        push!(matrixSUBGL, TSUBGL[searchsortedfirst((FS .- SUBGL[1:max_SUBGL]) ./ (FS), eps, rev=true)])
        end
    end
    return (matrixBLM), (matrixBPLM), (matrixSUBG), (matrixSGEP), (matrixDA), (matrixDWG), (matrixFGM), (matrixSUBGL), (matrixCG)
end

load_californian_time (generic function with 1 method)

In [7]:
# LOAD
BelgianBLM, BelgianBPLM, BelgianSUBG, BelgianSUBGEP, BelgianDA, BelgianDOWG, BelgianFGM, BelgianSUBGL, BelgianCG = load_belgian_time(5e-6)
CalifornianBLM, CalifornianBPLM, CalifornianSUBG, CalifornianSUBGEP, CalifornianDA, CalifornianDOWG, CalifornianFGM, CalifornianSUBGL, CalifornianCG = load_californian_time(5e-6)

(Any[74.58470179300002, 74.21409201700003, 74.43663895100002, 74.179379364, 900.0019076469995, 900.8868181089991, 900.1421035169999, 900.7148791899995, 122.24220301299998, 122.57561849399995, 121.57467175699999, 123.65630864199998, 438.26187291600013, 312.72115166699996, 310.704727128, 259.7372338759999], Any[258.191597594, 384.71187115099974, 152.60527002199998, 197.69563453799998, 737.7982109880003, 563.2000739649997, 561.2119245549999, 863.899574471, 184.622389787, 195.73090480200005, 161.377257578, 167.28619066599992, 534.212402288, 408.81523985800004, 452.423707088, 482.4752111929999], Any[126.39199268, 117.43362764899999, 119.498069242, 128.100817045, 444.385701368, 289.35272432100004, 130.77374846599994, 129.64347622700004, 18.672725291, 25.801031661, 25.48160004, 19.105642438, 783.4125225779998, 904.409931913, 902.7445662550001, 903.7320439390004], Any[20.890180234, 21.064697705, 20.850068613000005, 20.850960118999996, 112.04703265199998, 139.95576027300004, 112.13884860700001,

In [10]:
ds = BelgianCG
for inst in ds
    @show round((inst), sigdigits = 3)
end
@show round(mean(ds), sigdigits = 3)

round(inst, sigdigits = 3) = 450.0
round(inst, sigdigits = 3) = 414.0
round(inst, sigdigits = 3) = 443.0
round(inst, sigdigits = 3) = 565.0
round(inst, sigdigits = 3) = 422.0
round(inst, sigdigits = 3) = 523.0
round(inst, sigdigits = 3) = 429.0
round(inst, sigdigits = 3) = 322.0
round(mean(ds), sigdigits = 3) = 446.0


446.0